<a href="https://colab.research.google.com/github/xjdeng/github-ai-query/blob/main/download_repo_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install path.py==12.0.1
from path import Path as path
import os
from google.colab import files

def generate_tree(directory, prefix=""):

    tree = []
    entries = sorted(os.listdir(directory))  # Sort entries for consistent order

    for index, entry in enumerate(entries):
        path = os.path.join(directory, entry)
        is_last = (index == len(entries) - 1)

        if is_last:
            tree.append(f"{prefix}└── {entry}")
            next_prefix = f"{prefix}    "
        else:
            tree.append(f"{prefix}├── {entry}")
            next_prefix = f"{prefix}│   "

        if os.path.isdir(path):
            tree.append(generate_tree(path, next_prefix))
    return "\n".join(tree)

def get_repo(repo_url, refresh = False):
  if repo_url.lower().endswith(".git"):
    repo_url = repo_url[:-4]
  local_repo_dir = str(path(repo_url).name)
  if refresh:
    !rm -rf $local_repo_dir
  if not path(local_repo_dir).exists():
    !git clone $repo_url
  files_to_open = []
  for f in path(local_repo_dir).walkfiles():
    if (f.ext == ".py") or (f.ext == ".md") or (f.ext == ".txt"):
      files_to_open.append(f)
  prompt = f"""

  File structure:
  ---
  {generate_tree(local_repo_dir)}
  ---

  Next, I'll give you the contents of each file in the following section -
  *************************************************************************

  """
  for f in files_to_open:
    with open(f,'r') as ff:
      contents = ff.read()
    prompt += f"""
    {str(f.name)}:
    ---
    {contents}
    ---

    """
  prompt += "*************************************************************************"
  outfile = f"{local_repo_dir}.txt"
  with open(outfile,'w') as f:
    f.write(prompt)
  files.download(outfile)


In [2]:
get_repo("https://github.com/run-llama/llama_index")

Cloning into 'llama_index'...
remote: Enumerating objects: 110685, done.
remote: Counting objects: 100% (6746/6746), done.
remote: Compressing objects: 100% (387/387), done.
remote: Total 110685 (delta 6490), reused 6416 (delta 6335), pack-reused 103939 (from 1)
Receiving objects: 100% (110685/110685), 237.95 MiB | 24.47 MiB/s, done.
Resolving deltas: 100% (76667/76667), done.
Updating files: 100% (10765/10765), done.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>